# Time Series Forecasting: TSLA Stock

This notebook compares classical (ARIMA/SARIMA) and deep learning (LSTM) models for forecasting Tesla stock prices. Code is modular and OOP-based.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
sns.set(style='whitegrid')
%matplotlib inline

## OOP-based Data Preparation for Forecasting

In [ ]:
class TSLADataHandler:
    def __init__(self, start='2018-01-01', end=None):
        self.ticker = 'TSLA'
        self.start = start
        self.end = end
        self.data = None

    def load(self):
        self.data = yf.download(self.ticker, start=self.start, end=self.end, auto_adjust=True)['Close']
        return self.data

    def train_test_split(self, test_years=2):
        split_idx = int(len(self.data) * (1 - test_years / ((pd.to_datetime(self.data.index[-1]).year + 1) - pd.to_datetime(self.data.index[0]).year)))
        train, test = self.data.iloc[:split_idx], self.data.iloc[split_idx:]
        return train, test


## 1. Load and Split Data (Chronologically)

In [ ]:
handler = TSLADataHandler(start='2015-01-01')
tsla_close = handler.load()
train, test = handler.train_test_split(test_years=2)
print(f'Train: {train.index[0]} to {train.index[-1]}')
print(f'Test: {test.index[0]} to {test.index[-1]}')
plt.plot(train, label='Train')
plt.plot(test, label='Test')
plt.title('TSLA Closing Price: Train/Test Split')
plt.legend()
plt.show()

## 2. ARIMA/SARIMA Modeling

In [ ]:
# Use auto_arima for best (p,d,q)
arima_model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True, trace=True)
print(f'Best ARIMA order: {arima_model.order}')
arima_fit = ARIMA(train, order=arima_model.order).fit()
arima_forecast = arima_fit.forecast(steps=len(test))
plt.figure(figsize=(12,6))
plt.plot(train, label='Train')
plt.plot(test, label='Test')
plt.plot(test.index, arima_forecast, label='ARIMA Forecast')
plt.legend()
plt.title('ARIMA Forecast vs Actual')
plt.show()

In [ ]:
def evaluate_forecast(true, pred):
    mae = mean_absolute_error(true, pred)
    rmse = np.sqrt(mean_squared_error(true, pred))
    mape = np.mean(np.abs((true - pred) / true)) * 100
    return mae, rmse, mape

arima_mae, arima_rmse, arima_mape = evaluate_forecast(test, arima_forecast)
print(f'ARIMA MAE: {arima_mae:.2f}, RMSE: {arima_rmse:.2f}, MAPE: {arima_mape:.2f}%')

## 3. LSTM Modeling (Deep Learning)

In [ ]:
class LSTMForecaster:
    def __init__(self, lookback=30):
        self.lookback = lookback
        self.model = None
        self.scaler = MinMaxScaler()

    def prepare_data(self, series):
        scaled = self.scaler.fit_transform(series.values.reshape(-1,1))
        X, y = [], []
        for i in range(self.lookback, len(scaled)):
            X.append(scaled[i-self.lookback:i, 0])
            y.append(scaled[i, 0])
        X, y = np.array(X), np.array(y)
        X = X.reshape((X.shape[0], X.shape[1], 1))
        return X, y

    def build_model(self):
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(self.lookback, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        self.model = model
        return model

    def fit(self, X, y, epochs=20, batch_size=32):
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1)

    def forecast(self, last_sequence, n_steps):
        preds = []
        seq = last_sequence.copy()
        for _ in range(n_steps):
            pred = self.model.predict(seq.reshape(1, self.lookback, 1), verbose=0)[0,0]
            preds.append(pred)
            seq = np.roll(seq, -1)
            seq[-1] = pred
        return self.scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()


In [ ]:
lstm = LSTMForecaster(lookback=30)
X_train, y_train = lstm.prepare_data(train)
lstm.build_model()
lstm.fit(X_train, y_train, epochs=20, batch_size=32)
# Prepare last sequence from train+test for forecasting
full_series = pd.concat([train, test])
scaled_full = lstm.scaler.fit_transform(full_series.values.reshape(-1,1))
last_seq = scaled_full[len(train)-30:len(train)]
lstm_forecast = lstm.forecast(last_seq, n_steps=len(test))
plt.figure(figsize=(12,6))
plt.plot(train, label='Train')
plt.plot(test, label='Test')
plt.plot(test.index, lstm_forecast, label='LSTM Forecast')
plt.legend()
plt.title('LSTM Forecast vs Actual')
plt.show()

In [ ]:
lstm_mae, lstm_rmse, lstm_mape = evaluate_forecast(test, lstm_forecast)
print(f'LSTM MAE: {lstm_mae:.2f}, RMSE: {lstm_rmse:.2f}, MAPE: {lstm_mape:.2f}%')

## 4. Model Comparison & Discussion

| Model  | MAE  | RMSE | MAPE (%) |
|--------|------|------|----------|
| ARIMA  | {arima_mae:.2f} | {arima_rmse:.2f} | {arima_mape:.2f} |
| LSTM   | {lstm_mae:.2f} | {lstm_rmse:.2f} | {lstm_mape:.2f} |

- ARIMA is interpretable, fast to train, but may underperform on highly nonlinear series.
- LSTM can model nonlinearities and long-term dependencies, but requires more data and tuning.
- Choose based on business needs: interpretability vs. predictive power.
